In [ ]:
j = 0
x = None
codes = []
return_mat = np.zeros((11602, 1271))
j = 0
for i in code_shares:
    price_matrix= sp_data[i].astype('float')
    price_matrix = np.array(price_matrix)
    returns = np.log(price_matrix[1:]) - np.log(price_matrix[0:-1])
    if np.where(returns<-5)[0].shape[0] == 0:
        codes.append(i)
        returns[np.isnan(returns)]=-5
        return_mat[:, j] = returns
        j += 1
df_return = pd.DataFrame(data=return_mat, columns=codes)
df_return.to_csv('returns127.csv')

ffdata = pd.read_csv('./FF_Factors_daily.csv')
start_date = 19750102
end_date = 20201231
mask = (ffdata['date'] >= start_date) & (ffdata['date'] <= end_date)
ffdata = ffdata.loc[mask]
ffdata.to_csv('ffdata_11062.csv')

In [1]:
import json
import mgarch
import os
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from cvxopt import matrix
from cvxopt.blas import dot 
from cvxopt import solvers
solvers.options['show_progress'] = False


def load_json(path):
    with open(path, 'rb') as f:
        json_dict = json.load(f)
    return json_dict


def find_first_800(sp_data_matrix, num):
    posts = []
    trading_days = sp_data_matrix.shape[0]
    for i in range(trading_days):
        num_eff = np.sum(sp_data_matrix[i, 2:]>-5)
        if num_eff == num:
            posts.append(i)
    return posts[0]


def get_sub_mat(df_return, st_idx, code_shares, sp_data, sam_size):
    indx_length = sam_size
    num_lim = indx_length + 21
    ed_idx = st_idx + num_lim
    code_ls = list(df_return.columns)[1:]
    code_used = []
    res_mat = []
    res_dict = {}
    res_pre_dict = {}
    market_cap_dict = {}
    for code_i in code_ls:
        res = df_return[code_i][st_idx:ed_idx]
        eff_num = np.sum(res>-5)
        if eff_num == num_lim:
            cap_i = code_shares[code_i] * sp_data[code_i][st_idx+indx_length]# error: -1
            res_ls  = list(res)
            res_mat.append(res_ls[0:indx_length])
            code_used.append(code_i)
            res_dict[code_i] = res_ls[0:indx_length]
            res_pre_dict[code_i] = res_ls[indx_length:indx_length+21]
            market_cap_dict[code_i] = cap_i
    return res_mat, code_used, res_dict, res_pre_dict, market_cap_dict


def get_N(res_mat, code_used, res_dict, res_pre_dict, market_cap_dict, N):
    pos = np.where(np.corrcoef(np.array(res_mat))>0.95)
    if len(pos)>1:
        line_num, row_num = pos
        line_num = line_num.tolist()
        row_num = row_num.tolist()
        code_used_ori = copy.deepcopy(code_used)
        code_used = set(code_used)
        for i in range(len(line_num)):
            for j in range(len(row_num)):
                if  i > j:
                    stock_i = code_used_ori[line_num[i]]
                    stock_j = code_used_ori[row_num[j]]
                    cap_i = market_cap_dict[stock_i]
                    cap_j = market_cap_dict[stock_j]
            
                    if  cap_i>=cap_j and cap_i in code_used:
                        code_used.remove(stock_j)
                    elif cap_i<cap_j and cap_i in code_used:
                        code_used.remove(stock_i)
    
    code_cap = {}
    for code_i in code_used:
        code_cap[code_i] =  market_cap_dict[code_i]
        
    code_cap_sorted = sorted(code_cap.items(), key=lambda x: x[1], reverse=True)
    code_N = [i[0] for i in code_cap_sorted[0:N]]
    
    res_mat_new = []
    res_pre_new = []
    for code_i in code_N:
        res_mat_new.append(res_dict[code_i])
        res_pre_new.append(res_pre_dict[code_i])
    return res_mat_new, res_pre_new


def get_rolls(code_shares, sp_data, df_return, last_num, N, num, sam_size):
    st_id = find_first_800(df_return.to_numpy(), num)
    roll_len = sam_size+21
    j = 0
    all_rolls_res = 1
    while st_id+sam_size+21 <= last_num:
        res_mat, code_used, res_dict, res_pre_dict, market_cap_dict = get_sub_mat(df_return,
                                                st_id, code_shares, sp_data, sam_size)
        res_mat_new, res_pre_new = get_N(res_mat, code_used, 
                               res_dict, res_pre_dict, market_cap_dict, N)
        res_mat_new = np.array(res_mat_new).transpose()
        res_pre_new = np.array(res_pre_new).transpose()
        res_all = np.vstack((res_mat_new, res_pre_new))
        st_id += 21
        if all_rolls_res is  1:
            all_rolls_res = res_all
        else:
            all_rolls_res = np.vstack((all_rolls_res, res_all))
        
    return all_rolls_res


def get_return(weights, res_pre):
    weights = np.matrix(weights)
    res_pre = np.matrix(res_pre)
    return (weights*res_pre).tolist()[0]
    

def average_weights(n):
    return np.matrix([1/n for i in range(n)])


def get_ws_cov(cov, no_short=True):
    if no_short:
        N = cov.shape[0]
        P = matrix(cov)
        q = matrix(np.zeros((N, 1)))
        G  = matrix(-np.identity(N))
        h = matrix(np.zeros((N, 1)))
        A = matrix(1.0, (1, N))
        b = matrix(1.0)
        sol = solvers.qp(P, q, G, h, A, b)
        ws = np.matrix(sol['x']).T
    else:
        cov_mat = np.matrix(cov)
        N = cov_mat.shape[0]
        I = np.matrix(np.ones(N)).T
        ws = (cov_mat.I * I)/(I.T*cov_mat.I*I)
        ws = ws.T
    return ws  


def get_sam_res1(code_shares, sp_data, df_return, last_num, N, num, sam_size):
    st_id = find_first_800(df_return.to_numpy(), num)
    res_ew = []
    res_sample_cov = []
    while st_id + sam_size+21 <= last_num:
        res_mat, code_used, res_dict, res_pre_dict, market_cap_dict = get_sub_mat(df_return,
                                                                st_id, code_shares, sp_data, sam_size)
        res_mat_new, res_pre_new = get_N(res_mat, code_used, 
                               res_dict, res_pre_dict, market_cap_dict, N)
        st_id += 21
        avg_ws = average_weights(N)
        avg_res_i = get_return(avg_ws, res_pre_new)
        res_ew += avg_res_i
        sam_cov = np.cov(np.array(res_mat_new))
        samp_ws = get_ws_cov(sam_cov, no_short=False)
        sam_res_i = get_return(samp_ws,  res_pre_new)
        res_sample_cov += sam_res_i
    return res_ew, res_sample_cov


def indicators(res):
    AV= round(np.mean(res)*252*100, 5)
    SD = round(np.cov(res)**0.5*252**0.5*100, 5)
    IR = round(AV/SD, 5)
    return AV, SD, IR


def read_dcc_res(path):
    res_dccnl = pd.read_csv(path)
    res_dccnl= res_dccnl['x']
    res_dccnl = list(res_dccnl)
    return res_dccnl


def res_line(res):
    portfolio_res = [1]
    a = 1
    for i in res:
        a = a*(1+i)
        portfolio_res.append(a)
    return portfolio_res


def get_hold_ws(ws1, res_mat_pre):
    ws11 = np.multiply(ws1, (1+np.sum(res_mat_pre, axis=1)))
    ws11 = ws11/np.sum(ws11)
    return ws11.tolist()[0]


def indicators2(set_ws, hold_ws):
    set_ws = np.array(set_ws)
    hold_ws = np.array(hold_ws)
    ws_change = set_ws[1:, :] - hold_ws[0:-1, :]
    TO = round(np.mean(np.linalg.norm(ws_change, ord=1, axis=1)), 4)
    GL = round(np.mean(np.linalg.norm(set_ws, ord=1, axis=1)), 4)
    PL = round(np.mean(set_ws<0), 4)
    return TO, GL, PL


def get_ew_res(rolls, N, no_short=False):
    rtns_ew = []
    hold_ws = []
    set_ws = []
    for i in range(190):
        ws_i = average_weights(N)
        st_idx = (i+1)*1281-21
        ed_idx = (i+1)*1281
        res_pre_i = rolls[st_idx:ed_idx, 1:N+1]
        res_pre_i = np.matrix(res_pre_i).T
        rtns_i = get_return(ws_i, res_pre_i)
        hold_ws_i = get_hold_ws(ws_i, res_pre_i)
        set_ws.append(ws_i.tolist()[0])
        hold_ws.append(hold_ws_i)
        rtns_ew += rtns_i
    return rtns_ew, set_ws, hold_ws


def get_sam_res(rolls, N, no_short=False):
    rtns_sam = []
    hold_ws = []
    set_ws = []
    for i in range(190):
        st_idx = i*1281
        ed_idx = i*1281+1260
        res_mat = rolls[st_idx:ed_idx, 1:N+1]
        cov_mat =  np.cov(np.array(res_mat).transpose())
        ws_i = get_ws_cov(cov_mat, no_short=no_short)
        st_idx = (i+1)*1281-21
        ed_idx = (i+1)*1281
        res_pre_i = rolls[st_idx:ed_idx, 1:N+1]
        res_pre_i = np.matrix(res_pre_i).T
        rtns_sam_i = get_return(ws_i, res_pre_i)
        hold_ws_i = get_hold_ws(ws_i, res_pre_i)
        hold_ws.append(hold_ws_i)
        set_ws.append(ws_i.tolist()[0])
        rtns_sam += rtns_sam_i
    return rtns_sam, set_ws, hold_ws


def get_dcc_res(root_dir, rolls, N, no_short):
    file_ls = os.listdir(root_dir)
    length = len(file_ls)
    res_dccnl = []
    hold_ws = []
    set_ws = []
    for i in range(length):
        file_path = os.path.join(root_dir, '{}.csv'.format(i+1))
        cov_mat = pd.read_csv(file_path)
        cov_mat = np.matrix(cov_mat.to_numpy())
        #st_idx = i*1281
        #ed_idx = i*1281+1260
        #res_mat = rolls[st_idx:ed_idx, 1:N+1]
        #cov_mat =  np.cov(np.array(res_mat).transpose())
        ws_i = get_ws_cov(cov_mat, no_short=no_short)
        st_idx = (i+1)*1281-21
        ed_idx = (i+1)*1281
        res_pre_i = rolls[st_idx:ed_idx, 1:N+1]
        res_pre_i = np.matrix(res_pre_i).T
        res_i = get_return(ws_i, res_pre_i)
        hold_ws_i = get_hold_ws(ws_i, res_pre_i)
        hold_ws.append(hold_ws_i)
        set_ws.append(ws_i.tolist()[0])
        res_dccnl += res_i
    return res_dccnl, set_ws, hold_ws


def get_rtns_static5(rolls, N, no_short, resid5, param5, ff5_cov):
    rtns_static = []
    hold_ws = []
    set_ws = []
    for i in range(190):
        resid_i = resid5[i*500:i*500+N, :]
        cov_i = np.cov(resid_i)
        cov_i = np.diag(np.diag(cov_i))
        #cov_i[np.where(abs(cov_i)<1e-7)]=0.0
        param5_i = param5[i*500:i*500+N, :]
        ff_vol_i = ff5_cov[i*5:i*5+5, :]
        cov_i = np.matrix(cov_i)
        param5_i = np.matrix(param5_i)
        ff_vol_i = np.matrix(ff_vol_i)
        rtn_cov_i =  param5_i * ff_vol_i * param5_i.T + cov_i
        #st_idx = i*1281
        #ed_idx = i*1281+1260
        #res_mat = rolls[st_idx:ed_idx, 1:N+1]
        #cov_mat =  np.cov(np.array(res_mat).transpose())
        ws_i = get_ws_cov(rtn_cov_i, no_short=no_short)
        st_idx = (i+1)*1281-21
        ed_idx = (i+1)*1281
        res_pre_i = rolls[st_idx:ed_idx, 1:N+1]
        res_pre_i = np.matrix(res_pre_i).T
        res_i = get_return(ws_i, res_pre_i)
        hold_ws_i = get_hold_ws(ws_i, res_pre_i)
        hold_ws.append(hold_ws_i)
        set_ws.append(ws_i.tolist()[0])
        rtns_static += res_i
    return rtns_static, set_ws, hold_ws


def get_rtns_static1(rolls, N, no_short, resid1, param1, ff1_cov):
    rtns_static = []
    hold_ws = []
    set_ws = []
    for i in range(190):
        resid_i = resid1[i*500:i*500+N, :]
        cov_i = np.cov(resid_i)
        cov_i = np.diag(np.diag(cov_i))
        #cov_i[np.where(abs(cov_i)<1e-7)]=0.0
        param1_i = param1[i*500:i*500+N, :]
        ff_vol_i = ff1_cov[i, :]
        cov_i = np.matrix(cov_i)
        param1_i = np.matrix(param1_i)
        ff_vol_i = np.matrix(ff_vol_i)
        rtn_cov_i =  param1_i * ff_vol_i * param1_i.T + cov_i
        #st_idx = i*1281
        #ed_idx = i*1281+1260
        #res_mat = rolls[st_idx:ed_idx, 1:N+1]
        #cov_mat =  np.cov(np.array(res_mat).transpose())
        ws_i = get_ws_cov(rtn_cov_i, no_short=no_short)
        st_idx = (i+1)*1281-21
        ed_idx = (i+1)*1281
        res_pre_i = rolls[st_idx:ed_idx, 1:N+1]
        res_pre_i = np.matrix(res_pre_i).T
        res_i = get_return(ws_i, res_pre_i)
        hold_ws_i = get_hold_ws(ws_i, res_pre_i)
        hold_ws.append(hold_ws_i)
        set_ws.append(ws_i.tolist()[0])
        rtns_static += res_i
    return rtns_static, set_ws, hold_ws


def get_rtns_dcc_ff5(root_dir, rolls, N, param5, ff5_cov, no_short):
    file_ls = os.listdir(root_dir)
    length = len(file_ls)
    res_dccnl = []
    hold_ws = []
    set_ws = []
    for i in range(length):
        file_path = os.path.join(root_dir, '{}.csv'.format(i+1))
        resid_cov = pd.read_csv(file_path)
        resid_cov = np.matrix(resid_cov.to_numpy())
        param5_i = param5[i*500:i*500+N, :]
        ff_vol_i = ff5_cov[i*5:i*5+5, :]
        param5_i = np.matrix(param5_i)
        ff_vol_i = np.matrix(ff_vol_i)
        rtn_cov_i =  param5_i * ff_vol_i * param5_i.T + resid_cov
        #st_idx = i*1281
        #ed_idx = i*1281+1260
        #res_mat = rolls[st_idx:ed_idx, 1:N+1]
        #cov_mat =  np.cov(np.array(res_mat).transpose())
        ws_i = get_ws_cov(rtn_cov_i, no_short=no_short)
        st_idx = (i+1)*1281-21
        ed_idx = (i+1)*1281
        res_pre_i = rolls[st_idx:ed_idx, 1:N+1]
        res_pre_i = np.matrix(res_pre_i).T
        res_i = get_return(ws_i, res_pre_i)
        hold_ws_i = get_hold_ws(ws_i, res_pre_i)
        hold_ws.append(hold_ws_i)
        set_ws.append(ws_i.tolist()[0])
        res_dccnl += res_i
    return res_dccnl, set_ws, hold_ws


def get_rtns_dcc_ff1(root_dir, rolls, N, param1, ff1_cov, no_short):
    file_ls = os.listdir(root_dir)
    length = len(file_ls)
    res_dccnl = []
    hold_ws = []
    set_ws = []
    for i in range(length):
        file_path = os.path.join(root_dir, '{}.csv'.format(i+1))
        resid_cov = pd.read_csv(file_path)
        resid_cov = np.matrix(resid_cov.to_numpy())
        #resid_cov = np.diag(np.diag(resid_cov))
        param1_i = param1[i*500:i*500+N, :]
        ff_vol_i = ff1_cov[i, :]
        resid_cov = np.matrix(resid_cov)
        param5_i = np.matrix(param1_i)
        ff_vol_i = np.matrix(ff_vol_i)
        rtn_cov_i =  param1_i * ff_vol_i * param1_i.T + resid_cov
        #st_idx = i*1281
        #ed_idx = i*1281+1260
        #res_mat = rolls[st_idx:ed_idx, 1:N+1]
        #cov_mat =  np.cov(np.array(res_mat).transpose())
        ws_i = get_ws_cov(rtn_cov_i, no_short=no_short)
        st_idx = (i+1)*1281-21
        ed_idx = (i+1)*1281
        res_pre_i = rolls[st_idx:ed_idx, 1:N+1]
        res_pre_i = np.matrix(res_pre_i).T
        res_i = get_return(ws_i, res_pre_i)
        hold_ws_i = get_hold_ws(ws_i, res_pre_i)
        hold_ws.append(hold_ws_i)
        set_ws.append(ws_i.tolist()[0])
        res_dccnl += res_i
    return res_dccnl, set_ws, hold_ws

# 1 load_data and get return rates

In [3]:
code_shares = load_json('./code_shares.json')
sp_data = pd.read_csv('./SP1273_19750103_20201231.csv')
df_return = pd.read_csv('./returns1271.csv')

# 2 get all rolling res data

In [ ]:
all_rolls_res = get_rolls(code_shares, sp_data, df_return, 11602, 500, 800, 630)

In [315]:
all_rolls_res1 = np.around(all_rolls_res, decimals=6, out=None)
col_names = []
for i in range(500):
    col_names.append('asset{}'.format(i))
df_rolls = pd.DataFrame(data=all_rolls_res1, columns=col_names)
df_rolls.to_csv('df_rolls_500_630.csv')

# 3 get portfolio universe

In [2]:
rolls = pd.read_csv('./df_rolls.csv')
rolls = rolls.to_numpy()

# 4 structure free models

# N=100

In [3]:
N = 100
root_dir = './cov_{}_dcc/'.format(N)
rtns_dcc_100, set_ws_dcc_100,  hold_ws_dcc_100 = get_dcc_res(root_dir, rolls, N, no_short=False)
idx1_dcc_100 = indicators(rtns_dcc_100)
idx2_dcc_100 = indicators2(set_ws_dcc_100, hold_ws_dcc_100)

rtns_ew_100, set_ws_ew_100, hold_ws_ew_100 = get_ew_res(rolls, N)
idx1_ew_100 = indicators(rtns_ew_100)
idx2_ew_100 = indicators2(set_ws_ew_100, hold_ws_ew_100)

rtns_sam_100,  set_ws_sam_100, hold_ws_sam_100 = get_sam_res(rolls, N, False)
idx1_sam_100 = indicators(rtns_sam_100)
idx2_sam_100 = indicators2(set_ws_sam_100, hold_ws_sam_100)

In [4]:
print('EW', idx1_ew_100)
print('Sample', idx1_sam_100)
print('DCCNL',idx1_dcc_100)

EW (5.19047, 20.65605, 0.25128)
Sample (38.45129, 132.29348, 0.29065)
DCCNL (7.42985, 13.78978, 0.53879)


In [5]:
print('EW', idx2_ew_100)
print('Sample', idx2_sam_100)
print('DCCNL',idx2_dcc_100)

EW (0.9906, 1.0, 0.0)
Sample (444.9632, 441.713, 0.4336)
DCCNL (40.2612, 39.7631, 0.4646)


# N = 200

In [6]:
N = 200
root_dir = './cov_{}_dcc/'.format(N)
rtns_dcc_200, set_ws_dcc_200,  hold_ws_dcc_200 = get_dcc_res(root_dir, rolls, N, no_short=False)
idx1_dcc_200 = indicators(rtns_dcc_200)
idx2_dcc_200 = indicators2(set_ws_dcc_200, hold_ws_dcc_200)

rtns_ew_200, set_ws_ew_200, hold_ws_ew_200 = get_ew_res(rolls, N)
idx1_ew_200 = indicators(rtns_ew_200)
idx2_ew_200 = indicators2(set_ws_ew_200, hold_ws_ew_200)

rtns_sam_200,  set_ws_sam_200, hold_ws_sam_200 = get_sam_res(rolls, N, False)
idx1_sam_200 = indicators(rtns_sam_200)
idx2_sam_200 = indicators2(set_ws_sam_200, hold_ws_sam_200)

In [7]:
print('EW', idx1_ew_200)
print('Sample', idx1_sam_200)
print('DCCNL',idx1_dcc_200)

EW (5.58736, 20.94381, 0.26678)
Sample (-15.82624, 105.42171, -0.15012)
DCCNL (9.77188, 12.96225, 0.75387)


In [8]:
print('EW', idx2_ew_200)
print('Sample', idx2_sam_200)
print('DCCNL',idx2_dcc_200)

EW (0.9953, 1.0, 0.0)
Sample (516.6393, 513.3177, 0.4568)
DCCNL (14.6735, 14.5643, 0.4832)


# N=500

In [9]:
N = 500
root_dir = './cov_{}_dcc/'.format(N)
rtns_dcc_500, set_ws_dcc_500,  hold_ws_dcc_500 = get_dcc_res(root_dir, rolls, N, no_short=False)
idx1_dcc_500 = indicators(rtns_dcc_500)
idx2_dcc_500 = indicators2(set_ws_dcc_500, hold_ws_dcc_500)

rtns_ew_500, set_ws_ew_500, hold_ws_ew_500 = get_ew_res(rolls, N)
idx1_ew_500 = indicators(rtns_ew_500)
idx2_ew_500 = indicators2(set_ws_ew_500, hold_ws_ew_500)

rtns_sam_500,  set_ws_sam_500, hold_ws_sam_500 = get_sam_res(rolls, N, False)
idx1_sam_500 = indicators(rtns_sam_500)
idx2_sam_500 = indicators2(set_ws_sam_500, hold_ws_sam_500)

In [10]:
print('EW', idx1_ew_500)
print('Sample', idx1_sam_500)
print('DCCNL',idx1_dcc_500)

EW (6.82722, 21.95549, 0.31096)
Sample (7.43941, 12.60113, 0.59038)
DCCNL (8.1287, 12.13695, 0.66975)


In [11]:
print('EW', idx2_ew_500)
print('Sample', idx2_sam_500)
print('DCCNL',idx2_dcc_500)

EW (0.9981, 1.0, 0.0)
Sample (711.1294, 707.1217, 0.4707)
DCCNL (51.7719, 51.4227, 0.5014)


# 2 static model

## 2.1 five factor model

In [15]:
resid5 = np.load('./ff_model_data_1260/residual_mat5.npy')
param5 = np.load('./ff_model_data_1260/params_mat5.npy')
ff_cov5 = np.load('./ff_model_data_1260/ff5_vol.npy')

In [16]:
N = 100
no_short = False
rtns_static5_100, set_ws_static5_100, hold_ws_static5_100 = \
get_rtns_static5(rolls, N, no_short, resid5, param5, ff_cov5)


N = 200
no_short = False
rtns_static5_200, set_ws_static5_200, hold_ws_static5_200 = \
get_rtns_static5(rolls, N, no_short, resid5, param5, ff_cov5)


N = 500
no_short = False
rtns_static5_500, set_ws_static5_500, hold_ws_static5_500 = \
get_rtns_static5(rolls, N, no_short, resid5, param5, ff_cov5)

In [17]:
idx1_static5_100 = indicators(rtns_static5_100)
idx2_static5_100 = indicators2(set_ws_static5_100, hold_ws_static5_100)

idx1_static5_200 = indicators(rtns_static5_200)
idx2_static5_200 = indicators2(set_ws_static5_200, hold_ws_static5_200)

idx1_static5_500 = indicators(rtns_static5_500)
idx2_static5_500 = indicators2(set_ws_static5_500, hold_ws_static5_500)

## 2.2 one-factor model

In [18]:
resid1 = np.load('./ff_model_data_1260/residual_mat1.npy')
param1 = np.load('./ff_model_data_1260/params_mat1.npy')
ff_cov1 = np.load('./ff_model_data_1260/ff1_vol.npy')

In [19]:
N = 100
no_short = False
rtns_static1_100, set_ws_static1_100, hold_ws_static1_100 = \
get_rtns_static1(rolls, N, no_short, resid1, param1, ff_cov1)

N = 200
no_short = False
rtns_static1_200, set_ws_static1_200, hold_ws_static1_200 = \
get_rtns_static1(rolls, N, no_short, resid1, param1, ff_cov1)

N = 500
no_short = False
rtns_static1_500, set_ws_static1_500, hold_ws_static1_500 = \
get_rtns_static1(rolls, N, no_short, resid1, param1, ff_cov1)

In [20]:
idx1_static1_100 = indicators(rtns_static1_100)
idx1_static1_200 = indicators(rtns_static1_200)
idx1_static1_500 = indicators(rtns_static1_500)

idx2_static1_100 = indicators2(set_ws_static1_100, hold_ws_static1_100)
idx2_static1_200 = indicators2(set_ws_static1_200, hold_ws_static1_200)
idx2_static1_500 = indicators2(set_ws_static1_500, hold_ws_static1_500)

In [21]:
print('idicators1', ('AV','SD','IR'))
print('static5_100', idx1_static5_100)
print('static5_200', idx1_static5_200)
print('static5_500', idx1_static5_500)
print('static1_100', idx1_static1_100)
print('static1_200', idx1_static1_200)
print('static1_500', idx1_static1_500)

idicators1 ('AV', 'SD', 'IR')
static5_100 (9.42504, 14.6334, 0.64408)
static5_200 (10.63565, 14.81366, 0.71796)
static5_500 (10.82981, 14.39245, 0.75246)
static1_100 (10.02253, 16.0114, 0.62596)
static1_200 (11.2105, 16.31887, 0.68697)
static1_500 (11.07083, 16.18345, 0.68408)


In [22]:
print('idicators2', ('TO','GL','PL'))
print('static5_100', idx1_static5_100)
print('static5_200', idx1_static5_200)
print('static5_500', idx1_static5_500)
print('static1_100', idx2_static1_100)
print('static1_200', idx2_static1_200)
print('static1_500', idx2_static1_500)

idicators2 ('TO', 'GL', 'PL')
static5_100 (9.42504, 14.6334, 0.64408)
static5_200 (10.63565, 14.81366, 0.71796)
static5_500 (10.82981, 14.39245, 0.75246)
static1_100 (2.2572, 2.2586, 0.4483)
static1_200 (2.403, 2.4009, 0.4619)
static1_500 (2.4481, 2.444, 0.4806)


# 3 DCC-NL approxiamate model

## 3.1 five factor model

In [23]:
param5 = np.load('./ff_model_data_1260/params_mat5.npy')
ff_cov5 = np.load('./ff_model_data_1260/ff5_vol_dcc.npy')

In [24]:
N = 100
root_dir = './ff_model_data_1260/resid_dcc_cov5_100/'
no_short=False
res_dccnl_ff5_100, set_ws_dccnl_ff5_100, hold_ws_dccnl_ff5_100 = \
get_rtns_dcc_ff5(root_dir, rolls, N, param5, ff_cov5, no_short)


N = 200
root_dir = './ff_model_data_1260/resid_dcc_cov5_200/'
no_short=False
res_dccnl_ff5_200, set_ws_dccnl_ff5_200, hold_ws_dccnl_ff5_200 = \
get_rtns_dcc_ff5(root_dir, rolls, N, param5, ff_cov5, no_short)

In [25]:
idx1_dccnl_ff5_100 =  indicators(res_dccnl_ff5_100)
idx2_dccnl_ff5_100 =  indicators2(set_ws_dccnl_ff5_100, hold_ws_dccnl_ff5_100)


idx1_dccnl_ff5_200 =  indicators(res_dccnl_ff5_200)
idx2_dccnl_ff5_200 =  indicators2(set_ws_dccnl_ff5_200, hold_ws_dccnl_ff5_200)

# 3.2 one-factor model

In [26]:
param1 = np.load('./ff_model_data_1260/params_mat1.npy')
ff_cov1 = np.load('./ff_model_data_1260/ff1_vol_dcc.npy')

In [796]:
N = 100
root_dir = './ff_model_data_1260/resid_dcc_cov1_100/'
no_short=False
res_dccnl_ff1_100, set_ws_dccnl_ff1_100, hold_ws_dccnl_ff1_100 = \
get_rtns_dcc_ff1(root_dir, rolls, N, param1, ff_cov1, no_short)

N = 200
root_dir = './ff_model_data_1260/resid_dcc_cov1_200/'
no_short=False
res_dccnl_ff1_200, set_ws_dccnl_ff1_200, hold_ws_dccnl_ff1_200 = \
get_rtns_dcc_ff1(root_dir, rolls, N, param1, ff_cov1, no_short)

In [798]:
idx1_dccnl_ff1_100 = indicators(res_dccnl_ff1_100)
idx2_dccnl_ff1_100 = indicators2(set_ws_dccnl_ff1_100, hold_ws_dccnl_ff1_100)

idx1_dccnl_ff1_200 = indicators(res_dccnl_ff1_200)
idx2_dccnl_ff1_200 = indicators2(set_ws_dccnl_ff1_200, hold_ws_dccnl_ff1_200)

In [809]:
N = 500
root_dir = './ff_model_data_1260/resid_dcc_cov1_500/'
no_short=False
res_dccnl_ff1_500, set_ws_dccnl_ff1_500, hold_ws_dccnl_ff1_500 = \
get_rtns_dcc_ff1(root_dir, rolls, N, param1, ff_cov1, no_short)

In [810]:
indicators(res_dccnl_ff1_500)

(4.84649, 5.19567, 0.93279)

In [811]:
indicators(res_dccnl_ff1_200[0:len(res_dccnl_ff1_500)])

(8.75168, 7.98978, 1.09536)

In [807]:
print('idicators1', ('AV','SD','IR'))
print('dccnl_ff5_100', idx1_dccnl_ff5_100)
print('dccnl_ff5_200', idx1_dccnl_ff5_200)
print('dccnl_ff1_100', idx1_dccnl_ff1_100)
print('dccnl_ff1_200', idx1_dccnl_ff1_200)

idicators1 ('AV', 'SD', 'IR')
dccnl_ff5_100 (6.7857, 14.46385, 0.46915)
dccnl_ff5_200 (8.00029, 13.02876, 0.61405)
dccnl_ff1_100 (9.26, 13.63718, 0.67903)
dccnl_ff1_200 (10.06905, 12.38824, 0.81279)


In [808]:
print('idicators2', ('TO','GL','PL'))
print('dccnl_ff5_100', idx2_dccnl_ff5_100)
print('dccnl_ff5_200', idx2_dccnl_ff5_200)
print('dccnl_ff1_100', idx2_dccnl_ff1_100)
print('dccnl_ff1_200', idx2_dccnl_ff1_200)

idicators2 ('TO', 'GL', 'PL')
dccnl_ff5_100 (630.7817, 623.7603, 0.427)
dccnl_ff5_200 (111.4408, 110.3599, 0.4569)
dccnl_ff1_100 (272.962, 268.7343, 0.4339)
dccnl_ff1_200 (254.613, 252.3522, 0.4567)
